# Llama2 70B Round Table

In [3]:
%cd /content/drive/MyDrive/NLP/semevaltask9

/content/drive/MyDrive/NLP/semevaltask9


In [4]:
import torch
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [5]:
GLOBAL_SEED = 255

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


env: CUBLAS_WORKSPACE_CONFIG=:4096:8


## Use hugging face chat

In [6]:
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00


In [8]:
from hugchat import hugchat
from hugchat.login import Login

# Log in to huggingface and grant authorization to huggingchat
sign = Login(mail, password)
cookies = sign.login()

# Save cookies to the local directory
cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

'./cookies_snapshot/baktash.ansari1381@gmail.com.json'

In [9]:
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())

In [10]:
# Create a new conversation with mixtral
chatbot.switch_llm(1)
id = chatbot.new_conversation()
chatbot.change_conversation(id)

### Load datasets

In [11]:
# Import Dataset
data = np.load("./Data/SP_train.npy", allow_pickle = True ).tolist()
SP_train = pd.DataFrame(data)
data = np.load("./Data/SP_test.npy", allow_pickle = True ).tolist()
SP_test = pd.DataFrame(data)
data = np.load("./Data/SP_test_answer.npy", allow_pickle = True ).tolist()
SP_test_answer = pd.DataFrame(data)

data = np.load("./Data/WP_train.npy", allow_pickle = True ).tolist()
WP_train = pd.DataFrame(data)
data = np.load("./Data/WP_test.npy", allow_pickle = True ).tolist()
WP_test = pd.DataFrame(data)
data = np.load("./Data/WP_test_answer.npy", allow_pickle = True ).tolist()
WP_test_answer = pd.DataFrame(data)

data = np.load("./Data/sentence_puzzle.npy", allow_pickle = True ).tolist()
SP = pd.DataFrame(data)
data = np.load("./Data/word_puzzle.npy", allow_pickle = True ).tolist()
WP = pd.DataFrame(data)

In [12]:
for index, row in SP_test.iterrows():
  SP_test.at[index, 'label'] = int(SP_test_answer.loc[index][1])

In [13]:
for index, row in WP_test.iterrows():
  WP_test.at[index, 'label'] = int(WP_test_answer.loc[index][1])

## Initial Round

### Create Prompt

In [14]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: """

In [15]:
def generate_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

In [16]:
print(generate_prompt(SP_test.loc[115], prompt))

Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: A professional fisherman caught 30 fish during a 5-day tournament. Each day, he caught three more fish than the day before. How many fish did the fisherman catch on the first day?
A : Two.
B : One.
C : Three.
D : None of above.



### Run inference

In [17]:
import time
llamaResultsR0 = []
for index, row in SP_test.iterrows():
  # if index + 1 < 66:
  #   continue
  query_result = chatbot.query(generate_prompt(row, prompt))
  print(index + 1)
  print(query_result)
  llamaResultsR0.append({"id" : index+1, "response" : query_result.text})
  print()
  time.sleep(2)

1
 Solution: A

Reasoning:
The statement "the hardworking farmer is the son of the lazy farmer" implies that the hardworking farmer was born to the lazy farmer. However, the statement "the lazy farmer is not the father of the hardworking farmer" means that the lazy farmer did not raise or mentor the hardworking farmer. Therefore, the only possible explanation is that the lazy farmer is the mother of the hardworking farmer. This is an unconventional family structure where the mother, who is usually expected to be nurturing and caring, is portrayed as lazy, while the son, who is expected to be cared for, is instead diligent and hardworking.

Confidence: 0.8

This solution is the most plausible explanation based on the given information. It satisfies both statements and provides a reasonable interpretation of the unusual relationship between the two farmers. However, there may be other possible explanations that could also fit the given constraints, which is why my confidence level is not

In [19]:
llamaResultsR0

[{'id': 1,
  'response': ' Solution: A\n\nReasoning:\nThe statement "the hardworking farmer is the son of the lazy farmer" implies that the hardworking farmer was born to the lazy farmer. However, the statement "the lazy farmer is not the father of the hardworking farmer" means that the lazy farmer did not raise or mentor the hardworking farmer. Therefore, the only possible explanation is that the lazy farmer is the mother of the hardworking farmer. This is an unconventional family structure where the mother, who is usually expected to be nurturing and caring, is portrayed as lazy, while the son, who is expected to be cared for, is instead diligent and hardworking.\n\nConfidence: 0.8\n\nThis solution is the most plausible explanation based on the given information. It satisfies both statements and provides a reasonable interpretation of the unusual relationship between the two farmers. However, there may be other possible explanations that could also fit the given constraints, which is

In [20]:
# save dataframe:
llamaR0 = pd.DataFrame(llamaResultsR0)
llamaR0.to_csv("./LlamaR0.csv")

In [21]:
llamaR0

,id,response
0,1,"Solution: A\n\nReasoning:\nThe statement ""the..."
1,2,Solution: C\n\nReasoning:\nThe clue that the ...
2,3,Solution: C\n\nReasoning:\nThe question state...
3,4,Solution: B\n\nReasoning:\nThe question state...
4,5,Solution: C\n\nReasoning:\nThe probability of...
...,...,...
115,116,Solution: B\n\nReasoning:\nLet's analyze the ...
116,117,Solution: C\n\nReasoning:\nThe question state...
117,118,"Solution: D\n\nReasoning:\nThe description ""H..."
118,119,Solution: B\n\nReasoning:\nThe question asks ...


In [23]:
# create structural data frame:
Structural_Llama_R0 = []

for index, item in enumerate(llamaResultsR0):
  print(index)
  print(item['response'].split('Solution: '))

  solution = item['response'].split('Solution: ')[1][0]
  reasoning = item['response'].split('Reasoning:\n')[1].split('Confidence:')[0]
  confidence = item['response'].split('Confidence: ')[1].split('\n')[0].strip()

  Structural_Llama_R0.append({"Solution" : solution, "Reasoning" : reasoning, "Confidence" : confidence})

Structural_Llama_R0

0
[' ', 'A\n\nReasoning:\nThe statement "the hardworking farmer is the son of the lazy farmer" implies that the hardworking farmer was born to the lazy farmer. However, the statement "the lazy farmer is not the father of the hardworking farmer" means that the lazy farmer did not raise or mentor the hardworking farmer. Therefore, the only possible explanation is that the lazy farmer is the mother of the hardworking farmer. This is an unconventional family structure where the mother, who is usually expected to be nurturing and caring, is portrayed as lazy, while the son, who is expected to be cared for, is instead diligent and hardworking.\n\nConfidence: 0.8\n\nThis solution is the most plausible explanation based on the given information. It satisfies both statements and provides a reasonable interpretation of the unusual relationship between the two farmers. However, there may be other possible explanations that could also fit the given constraints, which is why my confidence level is 

[{'Solution': 'A',
  'Reasoning': 'The statement "the hardworking farmer is the son of the lazy farmer" implies that the hardworking farmer was born to the lazy farmer. However, the statement "the lazy farmer is not the father of the hardworking farmer" means that the lazy farmer did not raise or mentor the hardworking farmer. Therefore, the only possible explanation is that the lazy farmer is the mother of the hardworking farmer. This is an unconventional family structure where the mother, who is usually expected to be nurturing and caring, is portrayed as lazy, while the son, who is expected to be cared for, is instead diligent and hardworking.\n\n',
  'Confidence': '0.8'},
 {'Solution': 'C',
  'Reasoning': 'The clue that the house is located in a rural area near a railroad track suggests that the noise from the passing trains might have been a factor in the deaths of Romeo and Juliet. The fact that glass shards and water were found on the floor indicates that a aquarium or fish tank

In [24]:
finalDF = pd.DataFrame(Structural_Llama_R0)

In [25]:
finalDF

,Solution,Reasoning,Confidence
0,A,"The statement ""the hardworking farmer is the s...",0.8
1,C,The clue that the house is located in a rural ...,0.9
2,C,"The question states that before your birthday,...",1.0
3,B,The question states that a crucial assistant c...,1.0
4,C,The probability of a person having an unlisted...,0.8
...,...,...,...
115,B,Let's analyze the information given in the pro...,1.0
116,C,The question states that Bob was working on a ...,1.0
117,D,"The description ""He has wed numerous women, bu...",1.0
118,B,The question asks how many legs are on the flo...,1.0


In [26]:
finalDF.to_csv('./Llama_R0.csv')

In [27]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(finalDF['Solution'].tolist()):
  print(mapping[item])

0
2
2
1
2
0
2
2
3
2
1
2
2
0
1
1
1
2
2
1
1
2
2
3
1
2
0
2
1
1
1
3
2
1
3
2
1
0
2
2
1
1
0
0
2
2
2
2
0
1
0
0
2
1
1
0
1
1
1
1
2
1
1
1
1
0
2
1
2
3
2
2
2
2
1
2
0
3
1
3
2
3
1
0
3
1
3
2
1
2
1
2
2
0
2
1
3
3
0
1
1
2
2
2
2
2
0
3
3
3
1
1
1
2
3
1
2
3
1
2
